In [1]:
import os
import pandas as pd

In [ ]:
import pandas as pd

# Load the data
df = pd.read_csv('../datasets/weather_8.csv')

df['time_of_day'] = df['valid_time_gmt'].str[11:13].astype(int)
# df.head()


# Create a DataFrame with all possible combinations of filename, day_of_year, and time_of_day
filenames = df['filename'].unique()
days_of_year = range(1, 366)
times_of_day = range(1, 25)
index = pd.MultiIndex.from_product([filenames, days_of_year, times_of_day], names=['filename', 'day_of_year', 'time_of_day'])
template = pd.DataFrame(index=index).reset_index()

# Merge the template with the original data to fill in missing values
merged = pd.merge(template, df, how='left', on=['filename', 'day_of_year', 'time_of_day'])

# Forward-fill missing values
merged = merged.groupby('filename').apply(lambda x: x.ffill())

# Save the result to a new file
merged.to_csv('../datasets/weather_9.csv', index=False)

In [39]:
#read df head
import datetime


file_path = '../datasets/weather_9.csv'
df = pd.read_csv(file_path)
df.head()

#add new column time_of_day; 0-23, round down to floor hour
if 'time_of_day' not in df.columns:
    df['time_of_day'] = df['valid_time_gmt'].str[11:13].astype(int)
df.head()

#fill in missing valid_time_gmt with assuming year is 2015, use columns 'day_of_year' (0-364) and 'time_of_day' (0-23)
#to calculate valid_time_gmt

year = 2015


#convert day_of_year and time_of_day to float
df['day_of_year'] = df['day_of_year'].astype(float)
df['time_of_day'] = df['time_of_day'].astype(float)

#fill in missing values; lambda function to calculate valid_time_gmt
df['valid_time_gmt'] = df['valid_time_gmt'].fillna(df.apply(lambda row: datetime.datetime(year, 1, 1) + datetime.timedelta(days=row['day_of_year'], hours=row['time_of_day']), axis=1))

df.head()



,filename,day_of_year,time_of_day,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,dewPt,heat_index,...,wc,wdir,wdir_cardinal,wspd,precip_hrly,uv_desc,feels_like,uv_index,clds,wx_phrase_simple
0,ABE.csv,1.0,1.0,2016-01-01 01:51:00,N,41.0,26.0,2600.0,27.0,41.0,...,34.0,280.0,W,12.0,0.0,Low,34.0,0.0,OVC,Cloudy
1,ABE.csv,1.0,2.0,2016-01-01 02:51:00,N,40.0,26.0,2600.0,27.0,40.0,...,33.0,280.0,W,12.0,0.0,Low,33.0,0.0,OVC,Cloudy
2,ABE.csv,1.0,3.0,2016-01-01 03:51:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,280.0,W,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
3,ABE.csv,1.0,4.0,2016-01-01 04:51:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
4,ABE.csv,1.0,5.0,2016-01-01 04:51:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy


In [50]:
#for each unique filename; log the filename that doesn't have 365 days of data
#and also log the filename that doesnt have 24 hours of data for each day

#group by filename
grouped = df.groupby('filename')

#log file
log_file = open('weather_prep_log.txt', 'w')

full_success = True

#loop through each group
for name, group in grouped:

    #check if each group has 365 days of data
    if len(group['day_of_year'].unique()) != 365:
        log_file.write(f'{name} days: {len(group["day_of_year"].unique())}/{365} \n')
        full_success = False
        continue

    #check if each group has 24 hours of data for each day
    for day in group['day_of_year'].unique():
        if len(group.loc[group['day_of_year'] == day]['time_of_day'].unique()) != 24:
            log_file.write(f'{name} day {day} hours: {len(group.loc[group["day_of_year"] == day]["time_of_day"].unique())}/{24} \n')
            full_success = False
            continue

print('full_success: ' + str(full_success))
#close log file
log_file.close()

full_success: True


In [11]:
#list columns
df.columns

Index(['valid_time_gmt', 'day_ind', 'temp', 'wx_icon', 'icon_extd', 'dewPt',
       'heat_index', 'rh', 'pressure', 'vis', 'wc', 'wdir', 'wdir_cardinal',
       'wspd', 'precip_hrly', 'uv_desc', 'feels_like', 'uv_index', 'clds',
       'filename', 'wx_phrase_simple', 'day_of_year', 'time_of_day'],
      dtype='object')

In [51]:
#count nulls
df.isnull().sum()

filename                0
day_of_year             0
time_of_day             0
valid_time_gmt          0
day_ind                66
temp                   66
wx_icon                67
icon_extd              67
dewPt                  66
heat_index             66
rh                     66
pressure               98
vis                  8826
wc                     66
wdir                   66
wdir_cardinal          66
wspd                   66
precip_hrly         53885
uv_desc                66
feels_like             66
uv_index               66
clds                   66
wx_phrase_simple       66
dtype: int64

In [52]:
#override valid_time_gmt column with datetime object using year = 2015, day_of_year, and time_of_day
df['valid_time_gmt'] = df.apply(lambda row: datetime.datetime(year, 1, 1) + datetime.timedelta(days=row['day_of_year'], hours=row['time_of_day']), axis=1)

In [55]:
df.head(48)

,filename,day_of_year,time_of_day,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,dewPt,heat_index,...,wc,wdir,wdir_cardinal,wspd,precip_hrly,uv_desc,feels_like,uv_index,clds,wx_phrase_simple
0,ABE.csv,1.0,1.0,2015-01-02 01:00:00,N,41.0,26.0,2600.0,27.0,41.0,...,34.0,280.0,W,12.0,0.0,Low,34.0,0.0,OVC,Cloudy
1,ABE.csv,1.0,2.0,2015-01-02 02:00:00,N,40.0,26.0,2600.0,27.0,40.0,...,33.0,280.0,W,12.0,0.0,Low,33.0,0.0,OVC,Cloudy
2,ABE.csv,1.0,3.0,2015-01-02 03:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,280.0,W,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
3,ABE.csv,1.0,4.0,2015-01-02 04:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
4,ABE.csv,1.0,5.0,2015-01-02 05:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
5,ABE.csv,1.0,6.0,2015-01-02 06:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
6,ABE.csv,1.0,7.0,2015-01-02 07:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
7,ABE.csv,1.0,8.0,2015-01-02 08:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
8,ABE.csv,1.0,9.0,2015-01-02 09:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy
9,ABE.csv,1.0,10.0,2015-01-02 10:00:00,N,39.0,26.0,2600.0,27.0,39.0,...,34.0,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy


In [56]:
#fill all na with its previous value
df = df.fillna(method='ffill')

#check if there are any nulls
df.isnull().sum()

filename            0
day_of_year         0
time_of_day         0
valid_time_gmt      0
day_ind             0
temp                0
wx_icon             0
icon_extd           0
dewPt               0
heat_index          0
rh                  0
pressure            0
vis                 0
wc                  0
wdir                0
wdir_cardinal       0
wspd                0
precip_hrly         0
uv_desc             0
feels_like          0
uv_index            0
clds                0
wx_phrase_simple    0
dtype: int64

In [ ]:
#remove duplicates if they exist for 'filename', 'day_of_year', 'time_of_day'
df = df.drop_duplicates(subset=['filename', 'day_of_year', 'time_of_day'], keep='first')

In [59]:
#save to csv
df.to_csv('../datasets/weather_10.csv', index=False)